In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import os
os.chdir("/content/drive/My Drive/data")
import pandas as pd
import numpy as np
dat = pd.read_csv("final_data0713.csv", encoding='CP949')

In [123]:
variable = list(dat.columns)
variable.remove('y')
cat_features = list(dat.select_dtypes(include=['object', 'int64']))


In [124]:
! pip install catboost

In [125]:
from catboost import CatBoostRegressor

In [126]:
## 1월 31일
train = dat[dat['date']<'2020-01-24']
test = dat[('2020-01-24'<=dat['date']) & (dat['date']<'2020-01-31')]

In [127]:
train_x = train.loc[:, variable]
train_y = train.iloc[:, [0]]
test_x = test.loc[:, variable]
test_y = test.iloc[:, [0]]

In [128]:
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE')

In [129]:
cb_model.fit(train_x, train_y, 
             eval_set=(test_x,test_y), 
             cat_features=cat_features,
             use_best_model=True,
             verbose=True)

Learning rate set to 0.062326
0:	learn: 23.1116893	test: 20.5268335	best: 20.5268335 (0)	total: 9.63ms	remaining: 9.62s
1:	learn: 22.0461071	test: 19.7131638	best: 19.7131638 (1)	total: 27.9ms	remaining: 13.9s
2:	learn: 21.1501490	test: 18.9456562	best: 18.9456562 (2)	total: 34.1ms	remaining: 11.3s
3:	learn: 20.1981358	test: 18.0871768	best: 18.0871768 (3)	total: 47.9ms	remaining: 11.9s
4:	learn: 19.4008445	test: 17.3894534	best: 17.3894534 (4)	total: 56.1ms	remaining: 11.2s
5:	learn: 18.6227504	test: 16.6148895	best: 16.6148895 (5)	total: 68.7ms	remaining: 11.4s
6:	learn: 17.9153341	test: 16.1010235	best: 16.1010235 (6)	total: 78ms	remaining: 11.1s
7:	learn: 17.2548751	test: 15.5524181	best: 15.5524181 (7)	total: 88.7ms	remaining: 11s
8:	learn: 16.5930193	test: 14.8714774	best: 14.8714774 (8)	total: 100ms	remaining: 11s
9:	learn: 16.0196300	test: 14.4540824	best: 14.4540824 (9)	total: 111ms	remaining: 11s
10:	learn: 15.5043384	test: 14.0118127	best: 14.0118127 (10)	total: 121ms	remain

In [130]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
pred_y = cb_model.predict(test_x)
print("Model Evaluation Stage")
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_y, pred_y)))

Model Evaluation Stage
RMSE: 6.927563602050301


In [131]:
def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp =sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

In [132]:
true_y = test_y.to_numpy()
lossloss(true_y, pred_y)

0.2589101592936361

In [133]:
## 3월 31일
train = dat[dat['date']<'2020-03-24'].dropna()
test = dat[('2020-03-24'<=dat['date']) & (dat['date']<'2020-03-31')]

In [134]:
train_x = train.loc[:, variable]
train_y = train.iloc[:, [0]]
test_x = test.loc[:, variable]
test_y = test.iloc[:, [0]]

In [135]:
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE')

In [136]:
cb_model.fit(train_x, train_y, 
             eval_set=(test_x,test_y), 
             cat_features=cat_features,
             use_best_model=True,
             verbose=True)

Learning rate set to 0.065364
0:	learn: 24.0355452	test: 32.6586079	best: 32.6586079 (0)	total: 13.2ms	remaining: 13.2s
1:	learn: 22.9824027	test: 31.4574658	best: 31.4574658 (1)	total: 27.1ms	remaining: 13.5s
2:	learn: 21.9876091	test: 30.2854461	best: 30.2854461 (2)	total: 39.4ms	remaining: 13.1s
3:	learn: 21.0497792	test: 29.1860660	best: 29.1860660 (3)	total: 52.9ms	remaining: 13.2s
4:	learn: 20.2103729	test: 28.1997504	best: 28.1997504 (4)	total: 63.1ms	remaining: 12.6s
5:	learn: 19.3984871	test: 27.1421487	best: 27.1421487 (5)	total: 74.3ms	remaining: 12.3s
6:	learn: 18.6721648	test: 26.2863165	best: 26.2863165 (6)	total: 84.1ms	remaining: 11.9s
7:	learn: 17.9698764	test: 25.3254493	best: 25.3254493 (7)	total: 104ms	remaining: 12.9s
8:	learn: 17.3938195	test: 24.6512728	best: 24.6512728 (8)	total: 118ms	remaining: 13s
9:	learn: 16.8628242	test: 23.8948139	best: 23.8948139 (9)	total: 128ms	remaining: 12.7s
10:	learn: 16.3492262	test: 23.0004165	best: 23.0004165 (10)	total: 143ms	r

In [137]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
print("Model Evaluation Stage")
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_y, cb_model.predict(test_x))))


Model Evaluation Stage
RMSE: 7.596198076954024


In [138]:
true_y = test_y.to_numpy()
lossloss(true_y, pred_y)

0.42678102078126995

In [139]:
## 5월 31일
train = dat[dat['date']<'2020-05-24'].dropna()
test = dat[('2020-05-24'<=dat['date']) & (dat['date']<'2020-05-31')]

In [140]:
train_x = train.loc[:, variable]
train_y = train.iloc[:, [0]]
test_x = test.loc[:, variable]
test_y = test.iloc[:, [0]]

In [141]:
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE')

In [142]:
cb_model.fit(train_x, train_y, 
             eval_set=(test_x,test_y), 
             cat_features=cat_features,
             use_best_model=True,
             verbose=True)

Learning rate set to 0.067935
0:	learn: 25.7474884	test: 32.0879042	best: 32.0879042 (0)	total: 19.7ms	remaining: 19.7s
1:	learn: 24.4630057	test: 30.5986016	best: 30.5986016 (1)	total: 36.5ms	remaining: 18.2s
2:	learn: 23.2963438	test: 29.2245252	best: 29.2245252 (2)	total: 54.1ms	remaining: 18s
3:	learn: 22.2174587	test: 27.8841518	best: 27.8841518 (3)	total: 68.3ms	remaining: 17s
4:	learn: 21.2801080	test: 26.7643260	best: 26.7643260 (4)	total: 79.6ms	remaining: 15.8s
5:	learn: 20.4168735	test: 25.6802503	best: 25.6802503 (5)	total: 94.9ms	remaining: 15.7s
6:	learn: 19.5696035	test: 24.5731285	best: 24.5731285 (6)	total: 109ms	remaining: 15.5s
7:	learn: 18.7748262	test: 23.5238235	best: 23.5238235 (7)	total: 123ms	remaining: 15.3s
8:	learn: 17.9568562	test: 22.5124385	best: 22.5124385 (8)	total: 138ms	remaining: 15.2s
9:	learn: 17.3487169	test: 21.6149377	best: 21.6149377 (9)	total: 156ms	remaining: 15.4s
10:	learn: 16.7437520	test: 20.8139063	best: 20.8139063 (10)	total: 170ms	rema

In [143]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
print("Model Evaluation Stage")
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_y, cb_model.predict(test_x))))

Model Evaluation Stage
RMSE: 9.051421236635935


In [144]:
true_y = test_y.to_numpy()
lossloss(true_y, pred_y)

0.4032836796619496